In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd
def load_housing_data(housing_path = HOUSING_PATH):
  csv_path = os.path.join(housing_path,"housing.csv")
  return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
len(train_set),len(test_set) # 개발자가 여러명이기에 난수 발생에 제약을 걸어줘야 한다는 점 기억 시드를 넣는다. 처음 테스트 저장을 하거나
train_set[:3]

# 난수에 시드 삽입 예시

In [ ]:
import numpy as np

np.random.seed(42) # 매번 같은 데이터셋

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
len(train_set),len(test_set) #시드를 삽입했을 경우
train_set[:3]

# 데이터셋이 업데이트 되는 문제를 해결하는 방법

In [ ]:
housing.reset_index()

In [ ]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32 # 해쉬값 사용 8비트 정수형 값을 입력으로 넣어서 만들어줌 최대 값이 2에 32승 0.2를 곱해서 
    # 해쉬값을 계산하면 고르게 퍼지게 되어있음 0~20000개가 순차적으로 들어오면 퍼짐을 항상 균등하게 해준다. 
    # 데이터의 값에 비례하여 퍼지게 하기 위해 해쉬값을 사용한다. 0.2 보다 작으면 검증세트 크면 테스트 세트로 분리
    # 새로운 데이터가 들어왔을 때도 고정값으로 데이터를 퍼지게 하기 위해 사용한다.

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio)) # 각각의 요소에 대해서 일련의 처리를 하는 것이 apply에 lambda로 처리 이 결과를 test_set_check로 전달하여 밑 코드에 전달한다.
    return data.loc[~in_test_set], data.loc[in_test_set]

#여기까지 무작위 샘플링
#무작위 샘플링의 경우 극단적인 때 하나의 요소만 검증 세트나 훈련 세트에 편향되게 될 수 있는데 이를 샘플링 편향이라고 한다.
# 이를 계층적 샘플링으로 해결한다.
# 표본집단이 모집단의 특성을 반영하려면 그 수가 적더라도 모집단의 비율을 가져가야 한다.

In [ ]:
import hashlib

def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

# median_income 을 bins로 값을 바꿔준다. 해당 값들을 라벨링하여 이름을 붙여준다, 전체 데이터를 5개로 나눈것
# 원레의 데이터셋과 유사하게 바꿔주겠다는 작업
# 훈련 세트도 테스트 세트도 아래의 분포로 만들어 주겠다.

In [ ]:
housing["income_cat"].hist()
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]


# StratifiedShuffleSplit 클래스 사용 한번 돌면서 트레인과 테스트를 한번 반환 
# 데이를 인컴 캣으로 계층적 스플릿을 인덱스로 반환하라
# 데이터 셋 생성

In [ ]:
strat_test_set.head()

In [ ]:
 strat_test_set["income_cat"].value_counts() / len(strat_test_set) # 계층적 샘플링

In [ ]:
housing["income_cat"].value_counts() / len(housing) # 원레 카테고리

In [ ]:
from sklearn.model_selection import train_test_split

def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

# 임시로 만든 특성 삭제

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
# 특정 축을 지운다고 하면 특성 전체를 지운다
# 같은 세트를 불러와서 트레인과 테스트에 대해 드랍을 호출하게 되면 income_cat 특성을 날려준다.
# 판다스의 많은 메소드 들은 데이터 프레임의 메소드로 처리되어 있는데 
# 객체의 처리 값을 원본 객체에 그대로 반영한다. 반환값이 존재 하여 새로운 객체를 준다. inplace는 이런것이다.

#set_=set_.drop('income_cat',axis=1)과 같은 코드

In [ ]:
strat_test_set.head()

# 지리적 데이터 시각화

In [ ]:
# 파이썬 객체 특성 참조와 포인터의 특성을 띔
# a = [1,2,3]
# b = a
# b[0] = 5
# b = [5,2,3] == 이러면 a도 값이 바뀐다. 바뀌는 것이 싫다면 deep copy로 가야한다. 때문에 .copy()를 사용한다.

housing = strat_train_set.copy()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude",alpha=0.1) # alpha = 투명도
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.show()

#상관관계 보기

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

#9가지 특성을 가지고 있고 대칭적인 구조 대칭행렬
#우리는 주택가격에 대한 상관계수를 들여다 보는것이 의미가 있을 것이다.
#0에 가까울수록 관계가 없다는 것이다 음수여도 상관관계가 없는것이 아니라 -1 에 가까울수록 또 관계가 깊어지는 것이다.

In [ ]:
#시각화
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age","latitude"]
scatter_matrix(housing[attributes], figsize=(8, 8))